In [27]:
!which python

/Users/tareksanger/Library/Caches/pypoetry/virtualenvs/lumis-tzzr_5k5-py3.12/bin/python


In [28]:
from lumis.config import config
import os
import pprint

pp = pprint.PrettyPrinter(indent=4)

os.environ["OPENAI_API_KEY"] = config.openai_api_key

In [29]:
import logging
import sys
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.embeddings.langchain import LangchainEmbedding
from llama_index.embeddings.openai import OpenAIEmbedding


logging.basicConfig(
    stream=sys.stdout, level=logging.INFO
)  # logging.DEBUG for more verbose output


# define LLM
from llama_index.llms.openai import OpenAI
from llama_index.core import Settings

# from llama_index.embeddings.openai import OpenAIEmbedding

llm = OpenAI(temperature=0, model="gpt-4-turbo")
embed_model = OpenAIEmbedding(model="text-embedding-3-large")


Settings.llm = llm
Settings.embed_model = embed_model
Settings.chunk_size = 512

In [30]:
username = "neo4j"
password = "12345678"
address = "bolt://localhost:7687"
database = "neo4j"

In [33]:
from llama_index.core import StorageContext
from llama_index.graph_stores.neo4j import Neo4jGraphStore

graph_store = Neo4jGraphStore(
    username=username,
    password=password,
    url=address,
    database=database,
)
storage_context = StorageContext.from_defaults(graph_store=graph_store)

In [32]:
# graph_store.execute("CREATE TAG IF NOT EXISTS entity(name string)")
# graph_store.execute(
#     "CREATE EDGE IF NOT EXISTS relationship(relationship string)"
# )
# graph_store.execute("CREATE TAG INDEX entity_index ON entity(name(256))")
# CREATE TAG entity(name string);
# CREATE EDGE relationship(relationship string);
# CREATE TAG INDEX entity_index ON entity(name(256));

AttributeError: 'Neo4jGraphStore' object has no attribute 'execute'

In [34]:
from llama_index.core import Document
from llama_index.core.utils import globals_helper
from nltk.stem.porter import PorterStemmer


def process_document(document: Document):
  
  words = document.text.split(" ")
  words = [word for word in words]
  document.text = " ".join(words)
  return document

In [35]:
from llama_index.readers.wikipedia import WikipediaReader

loader = WikipediaReader()

documents = loader.load_data(
    pages=["Stantec"], auto_suggest=True,
)

# documents = [process_document(document) for document in documents]

In [39]:
from llama_index.core import KnowledgeGraphIndex
from llama_index.core.node_parser import SentenceSplitter

index = KnowledgeGraphIndex.from_documents(
    documents,
    storage_context=storage_context,
    max_triplets_per_chunk=100,
    include_embeddings=True,
)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


In [ ]:
query_engine = index.as_query_engine()


In [40]:
response = query_engine.query("Can you tell me what resources Stantec has it could utilize for future innovation?")

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:llama_index.core.indices.knowledge_graph.retrievers:> Querying with idx: ced69f5a-4c11-431e-a502-eb6bc0945bc6: Stantec Inc. is an international professional services company in the design ...
INFO:llama_index.core.indices.knowledge_graph.retrievers:> Querying with idx: 5e8c0f53-e260-4e85-9b13-319fa3b05198: “The move was a major achievement –– in a two-month period, we sought and rec...
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


In [41]:
import pprint

pprint.pprint(response.response)

('Stantec has a vast array of resources that could be utilized for future '
 'innovation. These include a global workforce of 28,000 employees, a presence '
 'in 400 locations across six continents, and a diverse range of services in '
 'fields such as engineering, architecture, environmental sciences, and '
 'project management. Additionally, Stantec has a history of acquiring over '
 '130 firms since 1994, which has expanded its expertise and capabilities in '
 "various sectors. The company's involvement in major projects like the Panama "
 'Canal Expansion and the Keystone Pipeline also demonstrates its capacity to '
 'handle large-scale and technologically complex projects. These resources, '
 'combined with its strategic acquisitions and global footprint, position '
 'Stantec well to leverage its diverse expertise for innovation in '
 'infrastructure and facilities projects worldwide.')
